In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv

NU_FIDUCIAL = 70.0e6 # 70 MHz
DURATION = 3600. # 3600 seconds
kB = 1.38e-16 # in cgs
cc = 3.0e10  # in cgs

# Radio Detectability of Hot Jupiters
For instruments GMRT, HERA, LOFAR, SKA, UTR-2, VLA, we want to estimate Sensitivity, Time, Resolution, Confusion Limit

The radiometer equation:
$$
\frac{\sigma}{1 \rm~Jy} = 10^{23} \frac{2 k}{\lambda^2} \frac{T_{\rm sys} \Omega_{\rm station}}{\sqrt{2 B t N^2/2}}
$$

where $B$ is the bandwidth, $t$ is the observing time, and $N$ is the number of antennas.

or, since 
$$\Omega \approx (\lambda / D)^2$$
we have
$$
\frac{\sigma}{1 \rm~Jy} \approx 
10^{23} \frac{2 k}{D^2} \frac{T_{\rm sys}}{\sqrt{2 B t N^2/2}}
$$

Note that the beam is actually
$$\Omega = \pi (1.22 \lambda / 2 D)^2 / \ln(2)$$
so the equation is
$$
\frac{\sigma}{1 \rm~Jy} = 
10^{23} \frac{2 k \, 1.22^2 \, \pi}{4 \ln{(2)} D^2} \frac{T_{\rm sys}}{\sqrt{2 B t N^2/2}}
$$
which is larger than the approximation by a factor of 1.69.

Galactic synchrotron dominates below 100MHz (Di Matteo+ 2002, Santos, Cooray, Knox 2005, McQuinn+ 2006).

Tsys from Rogers & Bowman (2008)  
Tsys = Tsky = 237K(nu/150MHz)**-2.52

In [2]:
def rms_Jy(observatory, tt, nu=70e6):
    """
    Returns the RMS noise in Jy for an observatory for a certain time and frequency.
    
    Default frequency is 70 MHz.
    """
    nn = observatory['num_antennas']
    num_baselines = nn * (nn - 1.) / 2.
    Tsys = 237 * (nu / 150e6)**(-2.52)
    lam = cc / nu
    bb = observatory['bandwidth_MHz'] * 1.0e6 # convert MHz to Hz
    DD = observatory['dish_diam_m'] * 100. # convert meters to cm
    area_eff = np.pi * (DD/2.)**2
    prefactor = 1e23 * 2. * 1.22**2 * np.pi / (4 *np.log(2)) * kB / area_eff
    return prefactor * Tsys / np.sqrt(2 * bb * tt * num_baselines)

In [3]:
# get table data
observatory_data = read_csv('telescope_data.csv')
observatory_data

,telescope,dish_diam_m,max_baseline_m,num_antennas,bandwidth_MHz
0,gmrt,45.00,25000,30,50
1,hera,14.00,1000,350,50
2,lofar_lba,35.00,100000,44,50
3,mwa,4.00,1000,128,50
4,ska,1.75,65000,130000,50
5,vla,25.00,36000,29,50
6,lwa,2.00,100000,256,50


In [4]:
# Add a column to observatory_data
observatory_data['rms_Jy'] = np.nan * observatory_data.num_antennas
#observatory_data

In [5]:
# We'll do the calculation assuming 70 MHz and 1 hr, with a bandwidth of 50 MHz
# that's a wavelength of 4.3 meters
cc_mks = cc / 100. # in mks
observatories = {}
this_nu = NU_FIDUCIAL
for ii, observatory in enumerate(observatory_data.telescope):
    this_observatory = {}
    for this_property in observatory_data:
        if this_property == 'telescope':
            continue
        this_observatory[this_property] = observatory_data[this_property].iloc[ii]
    bmax = this_observatory['max_baseline_m'] # in m
    theta = cc_mks / (this_nu * bmax)
    theta_asec = theta * 180. * 3600. / np.pi
    observatory_data.loc[ii, 'theta_res_asec'] = theta_asec
    observatory_data.loc[ii, 'rms_Jy'] = rms_Jy(this_observatory, DURATION, nu=this_nu)
    observatories[observatory] = this_observatory
#observatories
observatory_data

,telescope,dish_diam_m,max_baseline_m,num_antennas,bandwidth_MHz,rms_Jy,theta_res_asec
0,gmrt,45.00,25000,30,50,0.000378,35.359681
1,hera,14.00,1000,350,50,0.000330,883.992027
2,lofar_lba,35.00,100000,44,50,0.000424,8.839920
3,mwa,4.00,1000,128,50,0.011076,883.992027
4,ska,1.75,65000,130000,50,0.000057,13.599877
5,vla,25.00,36000,29,50,0.001269,24.555334
6,lwa,2.00,100000,256,50,0.022109,8.839920


In [6]:
# in mJy
observatory_data.rms_Jy * 1000.

0     0.378298
1     0.329849
2     0.424054
3    11.076217
4     0.056755
5     1.268704
6    22.108955
Name: rms_Jy, dtype: float64

# Confusion noise
Want 1 source per 10 beams (pixels)  
Perley & Ericson (1974)  
Wieringa (1991)

$$
\sigma^2 = \int_\Omega \int_0^{5 \sigma / P} s^2 n[s] B[\Omega]^2 P[\Omega]^2 ds d\Omega
$$

$s$: source flux  
$n$: # count at $s$   -- $k s^{-1.65}$ (close to 3/2) ... and $k \propto \nu^{-0.7}$  
$B$: synthesized beam  
$P$: primary beam  
$ds$: sources  
$d\Omega$: solid angle

Remove all sources above 5$\sigma$

At 150 MHz with MWA, for $P \sim 30^\circ$, $B \sim 3.4'$, we get $\sigma \sim 0.1 \rm~Jy$.

$b_{\rm max}$ is the maximum baseline

Should scale as
$$
\sigma^2 \propto \left( \frac{\lambda}{D} \right)^2  \left( \frac{\lambda}{b_{\rm max}} \right)^2 \, ,
$$
or
$$
\sigma \propto \lambda^2 ,
$$

so at 70 MHz should be a factor of 4 worse.

So,

$$
5\sigma \sim 0.1 {\rm~Jy} \times \left( \frac{\nu}{150 \rm~MHz} \right)^{-2} \left( \frac{D}{4 \rm~m} \right)^{-2} \left( \frac{b_{\rm max}}{1000 \rm~m} \right)^{-2}
$$

In the dense regime, the $D$ drops out.  In the sparse regime, the $D$ remains.

According to Jim Condon and Scott Ransom's upcoming book <i> Essential Radio Astronomy</i>, the confusion noise
changes between the low resolution regime ($\theta>10$ arcsec), and the resolved source counts ($\theta<10$ arcsec) regime:

$$
\sigma \approx 0.2 {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-0.7} \left( \frac{\theta}{1^{\prime}} \right)^2
~~~~~~ (\theta > 10^{\prime\prime})
$$
$$
\sigma \approx 2.2 {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-0.7} \left( \frac{\theta}{1^{\prime}} \right)^{10/3} 
~~~~~~ (\theta < 10^{\prime\prime})
$$

TODO:  
replace $\theta$ with $\lambda / D$ and express in $\nu$ and $D$, and multiply by 5 for crude estimate of $5\sigma$.  
Wait, should it actually be $\lambda / b_{\rm max}$??  
I'm going to go with that.

\begin{eqnarray}
5 \sigma & \sim {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-0.7} \left( \frac{\lambda / b_{\rm max}}{1^{\prime}} \frac{60^{\prime}}{1^\circ} \frac{180^\circ}{\pi \rm~radians} \right)^2 \\\
 & = 1.18 \times 10^7 {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-0.7} \left( \frac{\lambda}{b_{\rm max}} \right)^2 \\\
 & = 1.18 \times 10^7 {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-0.7} \left( \lambda \right)^2 \left( b_{\rm max} \right)^{-2} \\\
 & = 1.18 \times 10^7 {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-0.7} \left( c / \nu \right)^2 \left( b_{\rm max} \right)^{-2} \\\
 & = 1.06 \times 10^{24} {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-0.7} {\rm~m^2~s^{-2}} \left( \nu \right)^{-2} \left( b_{\rm max} \right)^{-2} \\\
 & = 1.06 \times 10^{24} {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-0.7} {\rm~m^2~s^{-2}} \left(10^9 \rm~Hz \right)^{-2} \left( \frac{\nu}{1 \rm~GHz} \right)^{-2} \left( b_{\rm max} \right)^{-2} \\\
 & = 1.06 \times 10^6 {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-0.7} {\rm~m^2} \left( \frac{\nu}{1 \rm~GHz} \right)^{-2} \left( b_{\rm max} \right)^{-2} \\\
 & = 1.06 \times 10^6 {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-2.7} {\rm~m^2} ({1000 \rm~m})^{-2} \left( \frac{b_{\rm max}}{1000 \rm~m} \right)^{-2} \\\
 & = 1.06 {\rm~mJy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-2.7} \left( \frac{b_{\rm max}}{1000 \rm~m} \right)^{-2} \\\
 & = 178 {\rm~mJy} \left( \frac{\nu}{150 \rm~MHz} \right)^{-2.7} \left( \frac{b_{\rm max}}{1000 \rm~m} \right)^{-2} \\\
 & = 1400 {\rm~mJy} \left( \frac{\nu}{70 \rm~MHz} \right)^{-2.7} \left( \frac{b_{\rm max}}{1000 \rm~m} \right)^{-2} \\\
\end{eqnarray}

At low frequency, the dense limit applies to most/all of our possible instruments:
GMRT, HERA, LOFAR, MWA, SKA, VLA, LWA

configuration A for VLA to maximize resolution.

The approximation seems to come from the Condon et al. 1998 (NVSS paper), which has a confusion limit of 0.011 mJy in a 17.5'' beam and a survey frequency of 1420 MHz (21 cm).  Thus the equation is:
$$
\sigma_c \approx 0.011 {\rm~mJy} \left( \frac{\nu}{1420 \rm~MHz} \right)^{-0.7} \left( \frac{\theta}{17.5''} \right)^2
$$


In [7]:
observatory_data['conf_mJy_per_beam'] = np.nan * observatory_data.num_antennas
observatory_data['conf_mJy_per_beam_highres'] = np.nan * observatory_data.num_antennas
observatory_data

,telescope,dish_diam_m,max_baseline_m,num_antennas,bandwidth_MHz,rms_Jy,theta_res_asec,conf_mJy_per_beam,conf_mJy_per_beam_highres
0,gmrt,45.00,25000,30,50,0.000378,35.359681,NaN,NaN
1,hera,14.00,1000,350,50,0.000330,883.992027,NaN,NaN
2,lofar_lba,35.00,100000,44,50,0.000424,8.839920,NaN,NaN
3,mwa,4.00,1000,128,50,0.011076,883.992027,NaN,NaN
4,ska,1.75,65000,130000,50,0.000057,13.599877,NaN,NaN
5,vla,25.00,36000,29,50,0.001269,24.555334,NaN,NaN
6,lwa,2.00,100000,256,50,0.022109,8.839920,NaN,NaN


In [8]:
def confusion_mJy(observatory, nu=70e6):
    bmax = observatory['max_baseline_m'] # in m
    answer_mJy = 1393. * (nu / 70e6)**(-2.7) * (bmax / 1000)**(-2)
    return answer_mJy

In [9]:
def confusion_mJy_highres(observatory, nu=70e6):
    c = 2.998e8 # m/s
    bmax = observatory['max_baseline_m'] # in m
    theta = c/(nu*bmax)
    theta_amin = theta * 180. * 60. / np.pi
    answer_mJy = 5.0 * 2.2 * (nu / 1000e6)**(-0.7) * (theta_amin)**(10./3.)
    return answer_mJy

In [10]:
for ii, observatory in enumerate(observatory_data.telescope):
    this_observatory = {}
    for this_property in observatory_data:
        if this_property == 'telescope':
            continue
        this_observatory[this_property] = observatory_data[this_property].iloc[ii]
    observatory_data.loc[ii,'conf_mJy_per_beam'] = confusion_mJy(this_observatory,nu=NU_FIDUCIAL)
    observatory_data.loc[ii,'conf_mJy_per_beam_highres'] = confusion_mJy_highres(this_observatory,nu=NU_FIDUCIAL)
observatory_data

,telescope,dish_diam_m,max_baseline_m,num_antennas,bandwidth_MHz,rms_Jy,theta_res_asec,conf_mJy_per_beam,conf_mJy_per_beam_highres
0,gmrt,45.00,25000,30,50,0.000378,35.359681,2.228800,12.116642
1,hera,14.00,1000,350,50,0.000330,883.992027,1393.000000,553582.426265
2,lofar_lba,35.00,100000,44,50,0.000424,8.839920,0.139300,0.119266
3,mwa,4.00,1000,128,50,0.011076,883.992027,1393.000000,553582.426265
4,ska,1.75,65000,130000,50,0.000057,13.599877,0.329704,0.501346
5,vla,25.00,36000,29,50,0.001269,24.555334,1.074846,3.593414
6,lwa,2.00,100000,256,50,0.022109,8.839920,0.139300,0.119266


In [11]:
#nu=70e6
#c=2.998e8
#bmax=25000
#theta=c/(nu*bmax)
